In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome('C:/Users/bdtol/Downloads/chromedriver_win32/chromedriver.exe' ,chrome_options=options)

C:\Users\bdtol\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


# QBR

## Player stats

In [3]:
final_titles=[]
player_url=[]
college_abr=[]
qbr=[]
paa=[]
plays=[]
epa=[]
passing =[]
run=[]
sack=[]
pen=[]
raw=[]
year = []
for i in range(2004,2020):
    
    url = "https://www.espn.com/college-football/qbr/_/season/{}".format(i)
    driver.get(url)
    
    #Clicks "Show More" on webpage twice
    c=0
    while c<2:
        more_buttons = driver.find_elements_by_class_name("AnchorLink.loadMore__link")
        for x in range(len(more_buttons)):
            if more_buttons[x].is_displayed():
                driver.execute_script("arguments[0].click();", more_buttons[x])
                time.sleep(1)
        c+=1
    page_source = driver.page_source
    
    #Scrape the full page  
    soup = BeautifulSoup(page_source, 'html.parser')
    names = soup.find('tbody', class_="Table__TBODY")

    #Player BIO
    title = names.findAll('div')
    final_titles += [title[i].find('a').next_element for i in range(len(title))]
    player_url += [title[i].find('a').attrs['href'] for i in range(len(title))]
    college_abr += [title[i].find('span').next_element for i in range(len(title))]
    year.append([i for x in range(len(title))])
    
    #Player Stats
    stats = soup.find_all('tbody', class_="Table__TBODY")
    player_stats = stats[1].findAll('tr')
    qbr += [player_stats[i].findAll('td')[0].next_element for i in range(len(player_stats))]
    paa += [player_stats[i].findAll('td')[1].next_element for i in range(len(player_stats))]
    plays += [player_stats[i].findAll('td')[2].next_element for i in range(len(player_stats))]
    epa += [player_stats[i].findAll('td')[3].next_element for i in range(len(player_stats))]
    passing += [player_stats[i].findAll('td')[4].next_element for i in range(len(player_stats))]
    run += [player_stats[i].findAll('td')[5].next_element for i in range(len(player_stats))]
    sack += [player_stats[i].findAll('td')[6].next_element for i in range(len(player_stats))]
    pen += [player_stats[i].findAll('td')[7].next_element for i in range(len(player_stats))]
    raw += [player_stats[i].findAll('td')[8].next_element for i in range(len(player_stats))]

In [4]:
#breakdown nested list
flatyear = []
for elem in year:
    flatyear.extend(elem)

In [5]:
df = pd.DataFrame([final_titles, college_abr, flatyear, qbr, paa, plays, epa, passing, run, sack, pen, raw])
df = pd.DataFrame.transpose(df)
df.columns = ['NAME', 'COLLEGE', 'YEAR', 'QBR', 'PAA', 'PLAYS', 'EPA', 'PASS', 'RUN', 'SACK', 'PEN', 'RAW']

In [6]:
df.head()

,NAME,COLLEGE,YEAR,QBR,PAA,PLAYS,EPA,PASS,RUN,SACK,PEN,RAW
0,Stefan LeFors,LOU,2004,86.3,68.3,351,66.8,66.5,2.8,-3.7,1.6,88.3
1,Alex Smith,UTAH,2004,84.5,75.8,480,73.4,63.5,16.0,-7.3,1.2,83.8
2,Jason White,OKLA,2004,84.1,56.6,454,61.2,70.9,-6.2,-6.1,2.7,78.5
3,Jason Campbell,AUB,2004,83.3,41.5,373,48.4,56.9,-3.9,-6.7,2.0,76.1
4,Matt Jones,ARK,2004,81.7,35.9,383,43.6,24.4,27.5,-7.7,-0.7,72.6


## Player bio

In [7]:
HTWT=[]
DOB=[]
draft=[]

for i in player_url:
    time.sleep(1)
    html_page = requests.get(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    r  = soup.findAll('div', class_='ttu')
    h=0
    d=0
    dr=0
    for e in r:
        if e.next_element == 'HT/WT':
            HTWT.append(e.next_element.next_element.next_element.next_element)
            h=1
        if e.next_element == 'DOB':
            DOB.append(e.next_element.next_element.next_element.next_element)
            d=1
        if e.next_element == 'Draft Info':
            draft.append(e.next_element.next_element.next_element.next_element)
            dr=1
            
    if h == 0:
        HTWT.append('na')
    if d == 0:
        DOB.append('na')
    if dr == 0:
        draft.append('na')

In [8]:
df2 = pd.DataFrame([HTWT, DOB, draft])
df2 = pd.DataFrame.transpose(df2)
df2.columns = ['HT/WT', 'DOB', 'DRAFT_STATUS']

In [9]:
df2.head()

,HT/WT,DOB,DRAFT_STATUS
0,na,6/7/1981,"2005: Rd 4, Pk 20 (CAR)"
1,"6' 4"", 215 lbs",5/7/1984 (36),"2005: Rd 1, Pk 1 (SF)"
2,na,na,na
3,na,12/31/1981,"2005: Rd 1, Pk 25 (WSH)"
4,na,4/22/1983,"2005: Rd 1, Pk 21 (JAX)"


In [10]:
df3 = pd.concat([df,df2], axis=1)

In [12]:
df3.to_csv('csv_files/player_stats.csv')

In [11]:
df3.head()

,NAME,COLLEGE,YEAR,QBR,PAA,PLAYS,EPA,PASS,RUN,SACK,PEN,RAW,HT/WT,DOB,DRAFT_STATUS
0,Stefan LeFors,LOU,2004,86.3,68.3,351,66.8,66.5,2.8,-3.7,1.6,88.3,na,6/7/1981,"2005: Rd 4, Pk 20 (CAR)"
1,Alex Smith,UTAH,2004,84.5,75.8,480,73.4,63.5,16.0,-7.3,1.2,83.8,"6' 4"", 215 lbs",5/7/1984 (36),"2005: Rd 1, Pk 1 (SF)"
2,Jason White,OKLA,2004,84.1,56.6,454,61.2,70.9,-6.2,-6.1,2.7,78.5,na,na,na
3,Jason Campbell,AUB,2004,83.3,41.5,373,48.4,56.9,-3.9,-6.7,2.0,76.1,na,12/31/1981,"2005: Rd 1, Pk 25 (WSH)"
4,Matt Jones,ARK,2004,81.7,35.9,383,43.6,24.4,27.5,-7.7,-0.7,72.6,na,4/22/1983,"2005: Rd 1, Pk 21 (JAX)"


# Power Index

In [309]:
team_name = []
over=[]
off =[]
defe=[]
spec =[]
year=[]
for i in range(2005,2020):
    url = "https://www.espn.com/college-football/fpi/_/view/efficiencies/season/{}".format(i)
    driver.get(url)
    driver.set_window_size(100, 100)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    body = soup.find_all('tbody', class_= 'Table__TBODY')
    teams = body[0].findAll('div')


    team_name += [teams[i].findAll('a')[1].next_element for i in range(len(teams))]
    year.append([i for x in range(len(teams))])
    
    stats = body[1].findAll('tr')
    over += [stats[i].findAll('td')[0].next_element.next_element for i in range(len(stats))]
    off += [stats[i].findAll('td')[2].next_element.next_element for i in range(len(stats))]
    defe += [stats[i].findAll('td')[4].next_element.next_element for i in range(len(stats))]
    spec += [stats[i].findAll('td')[6].next_element.next_element for i in range(len(stats))]

In [310]:
flatyear = []
for elem in year:
    flatyear.extend(elem)

In [311]:
power = pd.DataFrame([team_name, over, off, defe, spec, flatyear])
power = pd.DataFrame.transpose(power)
power.columns = ['COLLEGE', 'OVERALL', 'OFFENSE', 'DEFENSE', 'SPECIAL', 'YEAR']

In [339]:
power.to_csv('csv_files/power_index.csv')

# Passing stats

In [126]:
player_name=[]
year=[]
CMP=[]
ATT=[]
PCMP=[]
YDS=[]
AVG=[]
LNG=[]
TD=[]
INT=[]
SACK=[]
RTG=[]
for i in range(2004,2020):
    
    url = "https://www.espn.com/college-football/stats/player/_/season/{}".format(i)
    driver.get(url)
    
    #Clicks "Show More" on webpage three times
    c=0
    while c<3:
        more_buttons = driver.find_elements_by_class_name("AnchorLink.loadMore__link")
        for x in range(len(more_buttons)):
            if more_buttons[x].is_displayed():
                driver.execute_script("arguments[0].click();", more_buttons[x])
                time.sleep(1)
        c+=1
    page_source = driver.page_source
    
    #Scrape the full page  
    soup = BeautifulSoup(page_source, 'html.parser')
    body = soup.find_all('tbody', class_="Table__TBODY")
    
    names = body[0].findAll('div')
    player_name += [names[i].find('a').next_element for i in range(len(names))]
    year.append([i for x in range(len(names))])
    
    stats = body[1].findAll('tr')
    CMP += [stats[i].findAll('td')[1].next_element for i in range(len(stats))]
    ATT += [stats[i].findAll('td')[2].next_element for i in range(len(stats))]
    PCMP += [stats[i].findAll('td')[3].next_element for i in range(len(stats))]
    YDS += [stats[i].findAll('td')[4].next_element for i in range(len(stats))]
    AVG += [stats[i].findAll('td')[5].next_element for i in range(len(stats))]
    LNG += [stats[i].findAll('td')[6].next_element for i in range(len(stats))]
    TD += [stats[i].findAll('td')[7].next_element for i in range(len(stats))]
    INT += [stats[i].findAll('td')[8].next_element for i in range(len(stats))]
    SACK += [stats[i].findAll('td')[9].next_element for i in range(len(stats))]
    RTG += [stats[i].findAll('td')[10].next_element for i in range(len(stats))]

In [127]:
flatyear = []
for elem in year:
    flatyear.extend(elem)

In [132]:
passing = pd.DataFrame([player_name, flatyear, CMP, ATT, PCMP, YDS, AVG, LNG, TD, INT, SACK, RTG])
passing = pd.DataFrame.transpose(passing)
passing.columns = ['NAME', 'YEAR', 'CMP', 'PATT', 'CMP%', 'PYDS', 'PAVG', 'PLNG', 'PTD', 'INT', 'SACK', 'RTG']

In [133]:
passing.to_csv('csv_files/passing.csv')

# Rushing stats

In [13]:
player_name=[]
year=[]
POS=[]
RATT=[]
RYDS=[]
RAVG=[]
RLNG=[]
RTD=[]
for i in range(2004,2020):
    
    url = "https://www.espn.com/college-football/stats/player/_/stat/rushing/season/{}".format(i)
    driver.get(url)
    
    #Clicks "Show More" on webpage 50 times
    # Most rushing stats are RB and WR so more entries are needed to gather QB stats
    c=0
    while c<50:
        more_buttons = driver.find_elements_by_class_name("AnchorLink.loadMore__link")
        for x in range(len(more_buttons)):
            if more_buttons[x].is_displayed():
                driver.execute_script("arguments[0].click();", more_buttons[x])
                time.sleep(1)
        c+=1
    page_source = driver.page_source
    
    #Scrape the full page  
    soup = BeautifulSoup(page_source, 'html.parser')
    body = soup.find_all('tbody', class_="Table__TBODY")
    

    names = body[0].findAll('div')
    player_name += [names[i].find('a').next_element for i in range(len(names))]
    year.append([i for x in range(len(names))])
    
    stats = body[1].findAll('tr')
    POS += [stats[i].findAll('td')[0].next_element for i in range(len(stats))]
    RATT += [stats[i].findAll('td')[1].next_element for i in range(len(stats))]
    RYDS += [stats[i].findAll('td')[2].next_element for i in range(len(stats))]
    RAVG += [stats[i].findAll('td')[3].next_element for i in range(len(stats))]
    RLNG += [stats[i].findAll('td')[4].next_element for i in range(len(stats))]
    RTD += [stats[i].findAll('td')[5].next_element for i in range(len(stats))]

In [14]:
flatyear = []
for elem in year:
    flatyear.extend(elem)

In [15]:
rushing = pd.DataFrame([player_name, flatyear, POS,  RATT, RYDS, RAVG, RLNG, RTD])
rushing = pd.DataFrame.transpose(rushing)
rushing.columns = ['NAME', 'YEAR', 'POS', 'RATT', 'RYDS', 'RAVG', 'RLNG', 'RTD']

In [17]:
rushing.to_csv('csv_files/rushing.csv')

In [16]:
rushing.head()

,NAME,YEAR,POS,RATT,RYDS,RAVG,RLNG,RTD
0,J.J. Arrington,2004,[RB],289,"2,018",7.0,89,15
1,DeAngelo Williams,2004,[RB],313,"1,948",6.2,75,22
2,Adrian Peterson,2004,[RB],339,"1,925",5.7,80,15
3,Cedric Benson,2004,[RB],326,"1,834",5.6,60,19
4,Jamario Thomas,2004,[RB],285,"1,801",6.3,77,17
